In [17]:
import torch
torch.autograd.set_detect_anomaly(True)
if torch.cuda.is_available():
    print("GPU is available")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available")
    print("Using CPU")

GPU is not available
Using CPU


In [18]:
import torch
import torchvision.transforms as transforms
from PIL import Image


def load_image(image_path, max_size=250):
    image = Image.open(image_path).convert("RGB")
    max_dim = max(image.size)
    scale_factor = max_size / max_dim
    new_size = (int(image.size[0] * scale_factor), int(image.size[1] * scale_factor))
    image = image.resize(new_size, Image.LANCZOS)
    transform = transforms.ToTensor()
    image_tensor = transform(image).unsqueeze(0)
    return image_tensor

def create_image_pyramid(image, num_scales, r=4/3):
    image_pyramid = [image]
    for _ in range(num_scales - 1):
        h = image.shape[2]
        w = image.shape[3]
        new_size = (int(h / r), int(w / r))
        image = torch.nn.functional.interpolate(image, size=new_size, mode="bilinear", align_corners=False) 
        image_pyramid.append(image)
    return image_pyramid

def get_num_scales(image_size, min_dim=25, r=4/3):
    max_dim = max(image_size)
    num_scales = 0
    dim = max_dim
    while dim > min_dim:
        dim = dim / r
        num_scales += 1
    return num_scales

In [19]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, num_channels=3, n_kernels_per_block=32):
        super(Generator, self).__init__()
        self.conv_blocks = nn.Sequential(
            self._conv_block(num_channels*2, n_kernels_per_block),
            self._conv_block(n_kernels_per_block, n_kernels_per_block),
            self._conv_block(n_kernels_per_block, n_kernels_per_block),
            self._conv_block(n_kernels_per_block, n_kernels_per_block),
            nn.Conv2d(n_kernels_per_block, num_channels, kernel_size=3, padding=1)
        )

    def _conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, noise, upsampled_image):
        input_tensor = torch.cat((noise, upsampled_image), dim=1)
        residual_image = self.conv_blocks(input_tensor)
        output_image = upsampled_image + residual_image
        return output_image

In [20]:
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, num_channels=3, n_kernels_per_block=32):
        super(Discriminator, self).__init__()
        self.conv_blocks = nn.Sequential(
            self._conv_block(num_channels, n_kernels_per_block),
            self._conv_block(n_kernels_per_block, n_kernels_per_block),
            self._conv_block(n_kernels_per_block, n_kernels_per_block),
            self._conv_block(n_kernels_per_block, n_kernels_per_block),
            nn.Conv2d(n_kernels_per_block, 1, kernel_size=3, padding=1)
        )
    
    def _conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, x):
        out = self.conv_blocks(x)
        return out

In [21]:
import torch

def wgan_gp_loss(discriminator, real_images, generated_images, device, lambda_gp=10):
    batch_size = real_images.shape[0]

    real_scores = discriminator(real_images)
    real_scores = real_scores.mean()

    fake_scores = discriminator(generated_images)
    fake_scores = fake_scores.mean()

    




Training with 9 scales
Training scale: 9/9


c:\Users\Atharva\miniconda3\envs\singan\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Error detected in ConvolutionBackward0. Traceback of forward call that caused the error:
  File "c:\Users\Atharva\miniconda3\envs\singan\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Atharva\miniconda3\envs\singan\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Atharva\miniconda3\envs\singan\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Atharva\miniconda3\envs\singan\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Atharva\miniconda3\envs\singan\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\Atharva\miniconda3\envs\singan\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
    self.asynci

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 128, 3, 3]] is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!